In [1]:
import networkx as nx
import pandas as pd
import sys
import os.path
from os import path
from collections import defaultdict

In [2]:
scores_files = ["fair", "hybrid_node2vec", "dyadic_fair", "hybrid_balanced_node2vec", "node2vec", "fairwalk", "random"]
names = ["FREC", "E_FREC", "PREC", "E_PREC", "n2v", "FairWalk", "Random"]
def compute_stats(dataset):
    red_pr = pd.read_csv(dataset + "/red_absorbing_probabilities.csv")
    pr = pd.read_csv(dataset + "/out_pagerank.csv")
    com = pd.read_csv(dataset + "/out_community.txt", names=["Nodes", "Community"], sep='\\s+', skiprows=1)
    f = open(dataset + "/out_graph.txt")
    lines = f.readlines()
    f.close()
    G = nx.parse_edgelist(lines[1:], nodetype=int, create_using=nx.DiGraph)

    names = None
    if path.exists(dataset + "/names.txt"):
        names = pd.read_csv(dataset + "/names.txt", sep='\t')
#         print(names.info())

    # print(red_pr.info())
    # print(pr.info())
    # print(com.info())

    for sf in scores_files:
        print(sf)
        df = pd.read_csv(dataset + "/" + f"sc_{sf}.csv")
        pr_ = list()
        rpr_ = list()
        homophily = list()
        distance = list()
        no_exist = list()
        ratio = list()
        i = 0
        error = False
        nE = 0
        for index, row in df.iterrows():
            if row['Rounds'] == -1: continue
            edges = list(eval(str(row["Edges"])))
            for e in edges:
                F = False
                try:
                    try:
                        length = len(nx.shortest_path(G, source=e[1], target=e[0])) - 1
                        try:
                            comm = com.loc[com['Nodes'] == e[1]].iloc[0]["Community"]
                            ratio.append(comm) 
                        except:
                            F = True
                        if not F:
                            distance.append(length)
                            x = 0
                            neighs = [n for n in G.neighbors(e[1])]
                            for n in neighs:
                                if com.loc[com['Nodes'] == n].iloc[0]["Community"] == comm:
                                    x+=1    
                            if len(neighs) > 0:
                                homophily.append(float(x / len(neighs)))
                            else:
                                homophily.append(0)
                        try:
                            if not F:
                                pr_.append(round(pr[pr["Nodes"] == e[1]].iloc[0]["Pagerank"], 4))
                                rpr_.append(round(red_pr[red_pr["Nodes"] == e[1]].iloc[0]["Red Pagerank"], 4))
                        except:
                            F = True
                    except:
                        nE+=1
                except Exception:
                     pass
        data = list()
        for i in range(0, len(distance)):
            no_exist.append(nE)
        print(len(distance))
        print(len(no_exist))
        print(len(pr_))
        print(len(rpr_))
        print(len(homophily))
        print(len(ratio))
        y = dict(zip(["Distance", "Inf Dist", "PageRank", "Red PageRank", "Homophily", "Red Ratio"], 
                     [distance, no_exist, pr_, rpr_, homophily, ratio]))
        df = pd.DataFrame(y, columns = ["Distance", "Inf Dist", "PageRank", "Red PageRank", 
                                        "Homophily", "Red Ratio"])
        df.to_csv(f"{dataset}/{sf}_stats1.csv", index=False)

In [3]:
compute_stats("dblp_genre_5")

fair
16500
16500
16500
16500
16500
16500
hybrid_node2vec
16500
16500
16500
16500
16500
16500
dyadic_fair
16500
16500
16500
16500
16500
16500
hybrid_balanced_node2vec
16500
16500
16500
16500
16500
16500
node2vec
16500
16500
16500
16500
16500
16500
fairwalk
16500
16500
16500
16500
16500
16500
random
16497
16497
16497
16497
16497
16497


In [4]:
compute_stats("dblp_pub_5")

fair
16500
16500
16500
16500
16500
16500
hybrid_node2vec
16500
16500
16500
16500
16500
16500
dyadic_fair
16500
16500
16500
16500
16500
16500
hybrid_balanced_node2vec
16500
16500
16500
16500
16500
16500
node2vec
16500
16500
16500
16500
16500
16500
fairwalk
16500
16500
16500
16500
16500
16500
random
16500
16500
16500
16500
16500
16500


In [ ]:
compute_stats("books")

In [ ]:
compute_stats("blogs")

In [ ]:
compute_stats("twitter")

In [356]:
compute_stats("dblp_new_pubs")

fair
328
328
328
328
328
328
hybrid_node2vec
337
337
337
337
337
337
dyadic_fair
429
429
429
429
429
429
hybrid_balanced_node2vec
381
381
381
381
381
381
node2vec
1360
1360
1360
1360
1360
1360
fairwalk
5726
5726
5726
5726
5726
5726
random
961
961
961
961
961
961


In [357]:
compute_stats("dblp_new_gender")

fair
1739
1739
1739
1739
1739
1739
hybrid_node2vec
1415
1415
1415
1415
1415
1415
dyadic_fair
1231
1231
1231
1231
1231
1231
hybrid_balanced_node2vec
1012
1012
1012
1012
1012
1012
node2vec
1404
1404
1404
1404
1404
1404
fairwalk
6199
6199
6199
6199
6199
6199
random
954
954
954
954
954
954


In [5]:
def final(dataset):
    results = [[] for i in range(0, len(scores_files))]
    i = 0
    for sf in scores_files:
        df = pd.read_csv(dataset + "/" + f"{sf}_stats1.csv")
        mean = df.rename(columns={'Distance': 'Mean Distance', 'PageRank': 'Mean PageRank',
                                    'Red PageRank': 'Mean Red PageRank', 'Homophily': 'Mean Homophily'})
        median = df.rename(columns={'Distance': 'Median Distance', 'PageRank': 'Median PageRank',
                                    'Red PageRank': 'Median Red PageRank', 'Homophily': 'Median Homophily'})
        max = df.rename(columns={'Distance': 'Max Distance', 'PageRank': 'Max PageRank',
                                    'Red PageRank': 'Max Red Pagerank', 'Homophily': 'Max Homophily'})
        dfm = mean.mean()
        dfmed = median.median()
        dfmax = max.max()
    
        for j in range(0, 5):
            if j == 1:
                results[i].append(dfm.iloc[1]/len(dfm))
                continue
            results[i].append(dfm.iloc[j])
            results[i].append(dfmed.iloc[j])
            results[i].append(dfmax.iloc[j])
        results[i].append(dfm.iloc[5])
        i+=1
    
    y = dict(zip(names, results))
    df = pd.DataFrame(y)
    #print(results)
    df_tr = df.transpose()
    df_tr.rename(columns={0: "Mean Distance", 1: "Median Distance", 2: "Max Distance", 3: "Inf Dist", 
                          4: "Mean PageRank", 5: "Median Pagerank", 6: "Max PageRank", 7: "Mean Red PageRank", 
                          8: "Median Red Pagerank", 9: "Max Red PageRank", 10: "Mean Homophily", 
                          11: "Median Homophily", 12: "Max Homophily",  13: "Red Ratio"}, 
                 inplace=True)
    del df_tr["Inf Dist"]
    df_tr = df_tr.round(4)
    df_tr.insert(0, "Algo", names, True)
    return df_tr

In [6]:
final("dblp_genre_5")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,4.7348,5.0,5.0,0.0000,0.0000,0.0006,0.7251,0.7532,0.9387,0.8909,1.0000,1.0,0.9968
E_FREC,E_FREC,4.4817,5.0,5.0,0.0002,0.0002,0.0015,0.6375,0.6235,0.9387,0.7154,0.7586,1.0,0.9924
PREC,PREC,4.7681,5.0,5.0,0.0001,0.0001,0.0006,0.4606,0.6599,0.9387,0.9335,1.0000,1.0,0.6009
E_PREC,E_PREC,4.4386,5.0,5.0,0.0002,0.0002,0.0015,0.4210,0.5398,0.9387,0.7833,0.8443,1.0,0.5979
n2v,n2v,3.6902,4.0,5.0,0.0009,0.0008,0.0015,0.2127,0.1935,0.5770,0.7214,0.7586,1.0,0.0288
FairWalk,FairWalk,4.4242,5.0,5.0,0.0001,0.0001,0.0015,0.2434,0.2082,0.8557,0.6482,0.7222,1.0,0.2547
Random,Random,5.2858,5.0,13.0,0.0001,0.0000,0.0010,0.2485,0.2122,0.9387,0.6530,0.7250,1.0,0.2524


In [7]:
final("dblp_pub_5")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,4.3201,4.0,5.0,0.0001,0.0001,0.0007,0.3728,0.3678,0.5793,0.4331,0.4286,1.0,0.9352
E_FREC,E_FREC,4.2794,4.0,5.0,0.0001,0.0001,0.0015,0.3461,0.3373,0.5793,0.3255,0.3333,1.0,0.9327
PREC,PREC,4.3592,4.0,5.0,0.0001,0.0001,0.0006,0.2781,0.3363,0.5793,0.5805,0.5000,1.0,0.6908
E_PREC,E_PREC,4.2807,4.0,5.0,0.0001,0.0001,0.0015,0.2591,0.2801,0.5793,0.4963,0.4000,1.0,0.6883
n2v,n2v,4.0297,4.0,5.0,0.0004,0.0004,0.0015,0.0558,0.0516,0.3250,0.9217,0.9348,1.0,0.0005
FairWalk,FairWalk,4.1613,4.0,5.0,0.0001,0.0001,0.0015,0.0623,0.0423,0.5793,0.8839,1.0000,1.0,0.0777
Random,Random,5.2813,5.0,13.0,0.0001,0.0000,0.0015,0.0621,0.0386,0.5793,0.8828,1.0000,1.0,0.0803


In [380]:
final("books")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,3.4444,4.0,4.0,0.0118,0.0088,0.0263,0.9005,0.9486,0.9736,0.9195,1.000,1.0,0.9444
E_FREC,E_FREC,3.4111,4.0,4.0,0.0138,0.0114,0.0263,0.8955,0.9363,0.9736,0.9091,1.000,1.0,0.9444
PREC,PREC,3.9222,4.0,4.0,0.0138,0.0108,0.0283,0.6056,0.8064,0.9591,0.9074,1.000,1.0,0.6111
E_PREC,E_PREC,3.8667,4.0,4.0,0.0163,0.0166,0.0283,0.6025,0.8064,0.9591,0.8971,0.975,1.0,0.6111
n2v,n2v,2.9889,3.0,4.0,0.0202,0.0189,0.0283,0.5185,0.7581,0.9597,0.9324,1.000,1.0,0.5333
FairWalk,FairWalk,2.6333,2.0,4.0,0.0106,0.0085,0.0280,0.3920,0.0779,0.9704,0.9395,1.000,1.0,0.3667
Random,Random,3.2614,3.0,6.0,0.0103,0.0082,0.0280,0.3813,0.0756,0.9736,0.9492,1.000,1.0,0.3523


In [381]:
final("blogs")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,4.0234,4.0,7.0,0.0002,0.0002,0.0003,0.6221,0.6201,0.6392,1.0000,1.0000,1.0,1.0000
E_FREC,E_FREC,2.6935,3.0,5.0,0.0012,0.0014,0.0026,0.5436,0.5374,0.6392,0.9459,0.9571,1.0,1.0000
PREC,PREC,4.4005,4.0,7.0,0.0002,0.0002,0.0005,0.6203,0.6201,0.6392,1.0000,1.0000,1.0,0.9977
E_PREC,E_PREC,2.9117,3.0,5.0,0.0014,0.0014,0.0026,0.5305,0.5313,0.6392,0.9364,0.9524,1.0,0.9928
n2v,n2v,2.6457,2.0,6.0,0.0024,0.0018,0.0100,0.4479,0.5045,0.5653,0.8024,0.9231,1.0,0.8924
FairWalk,FairWalk,2.9143,3.0,5.0,0.0005,0.0003,0.0040,0.4467,0.5125,0.6140,0.8936,0.9954,1.0,0.8200
Random,Random,2.9172,3.0,7.0,0.0006,0.0003,0.0100,0.4412,0.5068,0.6392,0.8936,1.0000,1.0,0.7959


In [382]:
final("twitter")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,8.8733,9.0,14.0,0.0001,0.0000,0.0027,0.8353,0.8451,0.8771,0.9997,1.0,1.0,1.0000
E_FREC,E_FREC,7.5702,8.0,14.0,0.0004,0.0003,0.0027,0.8133,0.8178,0.8771,0.9984,1.0,1.0,0.9988
PREC,PREC,8.7421,9.0,14.0,0.0002,0.0001,0.0018,0.5140,0.3314,0.8554,0.9986,1.0,1.0,0.4796
E_PREC,E_PREC,8.0419,8.0,14.0,0.0005,0.0003,0.0027,0.5148,0.3594,0.8554,0.9973,1.0,1.0,0.4796
n2v,n2v,6.7230,7.0,13.0,0.0009,0.0007,0.0033,0.4966,0.3594,0.8178,0.9217,1.0,1.0,0.3798
FairWalk,FairWalk,4.0765,4.0,12.0,0.0001,0.0000,0.0033,0.5465,0.6222,0.8554,0.9539,1.0,1.0,0.5297
Random,Random,6.3267,6.0,12.0,0.0001,0.0000,0.0027,0.5338,0.4155,0.8451,0.9780,1.0,1.0,0.4904


In [383]:
final("dblp_new_pubs")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,4.2957,4.0,9.0,0.0,0.0,0.0001,0.4695,0.5468,0.5795,0.6260,0.7500,1.0,0.9268
E_FREC,E_FREC,4.2463,4.0,11.0,0.0,0.0,0.0001,0.4564,0.5081,0.5795,0.5984,0.7500,1.0,0.9288
PREC,PREC,4.3660,4.0,14.0,0.0,0.0,0.0001,0.2903,0.1595,0.5795,0.8075,1.0000,1.0,0.4545
E_PREC,E_PREC,4.3202,4.0,16.0,0.0,0.0,0.0001,0.3149,0.2924,0.5795,0.7484,0.8125,1.0,0.5223
n2v,n2v,4.4199,4.0,18.0,0.0,0.0,0.0004,0.1863,0.1844,0.5239,0.8355,0.8333,1.0,0.0287
FairWalk,FairWalk,3.8496,4.0,19.0,0.0,0.0,0.0002,0.2080,0.1845,0.5795,0.7387,0.8182,1.0,0.1390
Random,Random,6.1592,6.0,17.0,0.0,0.0,0.0001,0.1980,0.1823,0.4157,0.7901,0.8519,1.0,0.1093


In [384]:
final("dblp_new_gender")

,Algo,Mean Distance,Median Distance,Max Distance,Mean PageRank,Median Pagerank,Max PageRank,Mean Red PageRank,Median Red Pagerank,Max Red PageRank,Mean Homophily,Median Homophily,Max Homophily,Red Ratio
FREC,FREC,5.5980,5.0,18.0,0.0,0.0,0.0001,0.5399,0.5527,0.6813,0.8890,1.0000,1.0,0.9948
E_FREC,E_FREC,4.9767,4.0,17.0,0.0,0.0,0.0001,0.5149,0.5202,0.6813,0.8112,1.0000,1.0,0.9929
PREC,PREC,5.6101,5.0,17.0,0.0,0.0,0.0001,0.4943,0.5483,0.6231,0.9018,1.0000,1.0,0.8846
E_PREC,E_PREC,4.9891,4.0,18.0,0.0,0.0,0.0001,0.4741,0.4941,0.6231,0.8226,1.0000,1.0,0.8923
n2v,n2v,4.5684,4.0,16.0,0.0,0.0,0.0004,0.2988,0.3493,0.5202,0.4122,0.1667,1.0,0.6168
FairWalk,FairWalk,4.0555,4.0,16.0,0.0,0.0,0.0001,0.2600,0.2189,0.6323,0.5951,0.7391,1.0,0.2944
Random,Random,6.0493,6.0,19.0,0.0,0.0,0.0001,0.2487,0.2192,0.5930,0.6566,0.7368,1.0,0.2285


In [13]:
# print(final("books").to_latex(index=False))
# print(final("blogs").to_latex(index=False))
# print(final("twitter").to_latex(index=False))
# print(final("dblp_new_pubs").to_latex(index=False))
# print(final("dblp_new_gender").to_latex(index=False))
# print(final("dblp_genre_5").to_latex(index=False))
print(final("dblp_pub_5").to_latex(index=False))

\begin{tabular}{lrrrrrrrrrrrrr}
\toprule
     Algo &  Mean Distance &  Median Distance &  Max Distance &  Mean PageRank &  Median Pagerank &  Max PageRank &  Mean Red PageRank &  Median Red Pagerank &  Max Red PageRank &  Mean Homophily &  Median Homophily &  Max Homophily &  Red Ratio \\
\midrule
     FREC &         4.3201 &              4.0 &           5.0 &         0.0001 &           0.0001 &        0.0007 &             0.3728 &               0.3678 &            0.5793 &          0.4331 &            0.4286 &            1.0 &     0.9352 \\
   E\_FREC &         4.2794 &              4.0 &           5.0 &         0.0001 &           0.0001 &        0.0015 &             0.3461 &               0.3373 &            0.5793 &          0.3255 &            0.3333 &            1.0 &     0.9327 \\
     PREC &         4.3592 &              4.0 &           5.0 &         0.0001 &           0.0001 &        0.0006 &             0.2781 &               0.3363 &            0.5793 &          0.5805 &     

In [371]:
x = {"A": [1,2,3], "B": [2,3,4]}
algo_names = ["FREC", "E-FREC", "PREC", "E-PREC", "n2v", "FairWalk"]
df = pd.DataFrame(x, columns = ["A", "B"])
df
columns_ = [1, 2, 3 ,4 ,5, 6]
fruit_dictionary = dict(zip(algo_names, columns_))
# data = [{algo_names[i]: columns_[i]} for i in range(0, len(algo_names))]
fruit_dictionary

{'FREC': 1, 'E-FREC': 2, 'PREC': 3, 'E-PREC': 4, 'n2v': 5, 'FairWalk': 6}

In [8]:
pair_files = ["sc_fair.csv", "sc_hybrid_node2vec.csv", "sc_dyadic_fair.csv", "sc_hybrid_balanced_node2vec.csv",
              "sc_node2vec.csv", "sc_fairwalk.csv"]
algo_names = ["FREC", "E-FREC", "PREC", "E-PREC", "n2v", "FairWalk"]
def table_best_target_nodes(dataset, sep=" "):
    rounds = 10
    nodes = dict()
    columns_ = list()
    
    names = None
    if path.exists(dataset + "/names.txt"):
        names = pd.read_csv(dataset + "/names.txt", sep='\t')
    i = 0
    for sf in pair_files:
        dfs = dict()
        df = pd.read_csv(f"{dataset}/{sf}")
        edges = list()
        for r in range(1, rounds + 1):
            es = eval(df["Edges"][r])
            edges.extend(es)
        for e in edges:
            tgt = names[names['Node_id'] == e[1]].iloc[0]["Author_Name"]
            if tgt in dfs:
                dfs[tgt]+=1
            else:
                dfs[tgt] = 1
        dfs = dict(sorted(dfs.items(), key=lambda item: item[1], reverse=True))
        sorted_keys = sorted(dfs, key=dfs.get, reverse=True)
        col = list()
        c = 0
        for k in sorted_keys:
            col.append(k + f" ({str(dfs[k])})")
            c+=1
            if c==20:
                break
        columns_.append(col)
        i+=1
    y = dict(zip(algo_names, columns_))
    df = pd.DataFrame(y, columns = algo_names)
    print(df.to_latex(index=False))

In [9]:
table_best_target_nodes("dblp_genre_5")

\begin{tabular}{llllll}
\toprule
                     FREC &                         E-FREC &                  PREC &                         E-PREC &                        n2v &                    FairWalk \\
\midrule
       Yoon-Min Nam (500) &           Wook-Shin Han (1175) &    Yoon-Min Nam (311) &            Wook-Shin Han (767) &     Jiawei Han 0001 (1563) &         Hui Xiong 0001 (19) \\
     Donghyoung Han (499) &             Shou-De Lin (1108) &  Donghyoung Han (310) &              Shou-De Lin (710) &            Jian Pei (1544) &            Aoying Zhou (18) \\
        Yu-Chi Chen (462) &            Meeyoung Cha (1069) &     Yu-Chi Chen (303) &             Meeyoung Cha (690) &       Jie Tang 0001 (1533) &             Zhenhui Li (14) \\
          Jihye Lee (460) &           Niloy Ganguly (1026) &       Jihye Lee (286) &            Niloy Ganguly (676) &  Christos Faloutsos (1526) &        Jiawei Han 0001 (13) \\
     Kyongseok Park (450) &        Jae-Gil Lee 0001 (1004) &  Kyongs

In [11]:
table_best_target_nodes("dblp_pub_5")

\begin{tabular}{llllll}
\toprule
                     FREC &                    E-FREC &                      PREC &                      E-PREC &                                   n2v &                          FairWalk \\
\midrule
 Madhav Nimishakavi (853) &            Wenwu Ou (907) &  Madhav Nimishakavi (639) &              Wenwu Ou (719) &                  Jure Leskovec (1396) &                     Wei Chen (19) \\
      Sen Yang 0002 (652) &  Madhav Nimishakavi (856) &       Sen Yang 0002 (472) &    Madhav Nimishakavi (642) &                     Heng Huang (1245) &                Huan Liu 0001 (14) \\
       Brian Zylich (526) &     Federico Tomasi (668) &        Brian Zylich (380) &       Federico Tomasi (503) &          Elke A. Rundensteiner (1224) &                 Xingquan Zhu (13) \\
     Tabassum Kakar (497) &      Zhaoqiang Chen (634) &      Tabassum Kakar (364) &        Zhaoqiang Chen (496) &                  Sang-Wook Kim (1199) &                     Jian Pei (13) \\
   

In [52]:
table_best_target_nodes("dblp_new_pubs")

\begin{tabular}{llllll}
\toprule
                         FREC &                     E-FREC &                     PREC &                      E-PREC &                              n2v &              FairWalk \\
\midrule
         Xiaoqing Huang (937) &           Jincai Lai (935) &     Xiaoqing Huang (667) &            Jincai Lai (704) &         Daniel D. Corkill (1033) &   Qinghua Zheng (620) \\
               Pin Xiao (875) &       Xiaoqing Huang (935) &           Pin Xiao (617) &        Xiaoqing Huang (665) &             Amanda Gentzel (980) &        Lina Yao (421) \\
          Corbin Rosset (747) &            Haoyu Han (872) &      Corbin Rosset (519) &             Haoyu Han (622) &            Fotis Psallidas (970) &   Sen Wang 0001 (388) \\
         Gale Yan Huang (702) &             Pin Xiao (858) &     Gale Yan Huang (489) &              Pin Xiao (604) &             Lisa Friedland (943) &       Yang Chen (365) \\
              Haoyu Han (599) &         Tianqiao Liu (721) &       X

In [51]:
table_best_target_nodes("dblp_new_gender")

\begin{tabular}{llllll}
\toprule
                          FREC &                   E-FREC &                     PREC &                   E-PREC &                              n2v &              FairWalk \\
\midrule
             Limeng Cui (1444) &        Limeng Cui (1121) &         Limeng Cui (956) &         Limeng Cui (710) &           Fotis Psallidas (1041) &   Qinghua Zheng (780) \\
            Yi-Feng Lan (1257) &       Yi-Feng Lan (1058) &        Yi-Feng Lan (856) &        Yi-Feng Lan (704) &             Amanda Gentzel (940) &        Lina Yao (564) \\
           Hsu-Chao Lai (1058) &       Dongwon Lee (1004) &       Hsu-Chao Lai (715) &        Dongwon Lee (671) &             Lisa Friedland (896) &   Sen Wang 0001 (509) \\
            Dongwon Lee (1012) &  Wush Chi-Hsuan Wu (931) &        Dongwon Lee (679) &  Wush Chi-Hsuan Wu (624) &             Subru Krishnan (827) &       Yang Chen (468) \\
            Haeseung Seo (915) &       Haeseung Seo (915) &       Haeseung Seo (613) &  